In [1]:
import pickle 
import matplotlib.pyplot as plt
import numpy as np 

In [2]:
def load_pickle(name): 
    file = open(name, "rb")
    data = pickle.load(file)
    file.close()

    return data

In [3]:
parameter = {
    #"DmCO2": 5.3e-5, 
    "C1": 2.562434e-12, 
    "hgx": 25 * 1e-3, 
    #"Cp_sol": 1.457, 
    #"rho_sol": 1144
    "delH_1": 98.76,
    "delH_2": 77.11, 
    "delH_3": 21.25
}

step = 0.001

In [4]:
upper_ads_c1 = load_pickle("C1_upper_ads")
upper_des_c1 = load_pickle("C1_upper_des")
lower_ads_c1 = load_pickle("C1_lower_ads")
lower_des_c1 = load_pickle("C1_lower_des")

upper_ads_hgx = load_pickle("hgx_upper_ads")
upper_des_hgx = load_pickle("hgx_upper_des")
lower_ads_hgx = load_pickle("hgx_lower_ads")
lower_des_hgx = load_pickle("hgx_lower_des")

upper_ads_h1 = load_pickle("delH_1_upper_ads")
upper_des_h1 = load_pickle("delH_1_upper_des")
lower_ads_h1 = load_pickle("delH_1_lower_ads")
lower_des_h1 = load_pickle("delH_1_lower_des")

upper_ads_h2 = load_pickle("delH_2_upper_ads")
upper_des_h2 = load_pickle("delH_2_upper_des")
lower_ads_h2 = load_pickle("delH_2_lower_ads")
lower_des_h2 = load_pickle("delH_2_lower_des")

upper_ads_h3 = load_pickle("delH_3_upper_ads")
upper_des_h3 = load_pickle("delH_3_upper_des")
lower_ads_h3 = load_pickle("delH_3_lower_ads")
lower_des_h3 = load_pickle("delH_3_lower_des")

In [5]:
original_ads = load_pickle("original_ads")
original_des = load_pickle("original_des")

In [6]:
measurement_names = list(upper_ads_c1.keys())
parameter_names = list(parameter.keys())

print(measurement_names)

['y_CO2_0', 'y_CO2_1', 'y_CO2_2', 'C_CO2_0', 'C_CO2_1', 'C_CO2_2', 'C_N2_0', 'C_N2_1', 'C_N2_2', 'Tg_0', 'Tg_1', 'Tg_2', 'Pg_0', 'Pg_1', 'Pg_2', 'vg_0', 'vg_1', 'vg_2', 'qCO2_0', 'qCO2_1', 'qCO2_2', 'Ts_0', 'Ts_1', 'Ts_2', 'ytheta_0', 'ytheta_1', 'ytheta_2']


In [7]:
def gradient(upper, lower, original, step, param_value, option="scale_parameter"):
    if option=="scale_parameter":
        return (upper-lower)/2/step
    elif option=="not_scale_parameter":
        return (upper-lower)/2/step/param_value
    elif option=="scale_para_and_measure":
        return (upper/original-lower/original)/2/step

In [15]:
def Q_way2(key, adsorption=True, scale_option="scale_parameter", step=0.001, std=0.01):
    if adsorption:
        upper_key_c1 = upper_ads_c1[key]
        lower_key_c1 = lower_ads_c1[key]

        upper_key_hgx = upper_ads_hgx[key]
        lower_key_hgx = lower_ads_hgx[key]

        upper_key_h1 = upper_ads_h1[key]
        lower_key_h1 = lower_ads_h1[key]

        upper_key_h2 = upper_ads_h2[key]
        lower_key_h2 = lower_ads_h2[key]

        upper_key_h3 = upper_ads_h3[key]
        lower_key_h3 = lower_ads_h3[key]

        original = original_ads[key]

    else:
        upper_key_c1 = upper_des_c1[key]
        lower_key_c1 = lower_des_c1[key]

        upper_key_hgx = upper_des_hgx[key]
        lower_key_hgx = lower_des_hgx[key]

        upper_key_h1 = upper_ads_h1[key]
        lower_key_h1 = lower_ads_h1[key]

        upper_key_h2 = upper_ads_h2[key]
        lower_key_h2 = lower_ads_h2[key]

        upper_key_h3 = upper_ads_h3[key]
        lower_key_h3 = lower_ads_h3[key]

        original = original_des[key]

    print("Measurement: ", key)
    
    Q = [] # N*p, p is # of parameters, N is # of measurements  

    for i in range(len(upper_key_c1)):
        #print(i)
        Q_t = []
        
        #Q_t.append(gradient(upper_key_dmco2[i], lower_key_dmco2[i], original[i], step, parameter["DmCO2"], option=scale_option))
        Q_t.append(gradient(upper_key_c1[i], lower_key_c1[i],original[i],  step, parameter["C1"], option=scale_option))
        Q_t.append(gradient(upper_key_hgx[i], lower_key_hgx[i],original[i],  step, parameter["hgx"], option=scale_option))
        #Q_t.append(gradient(upper_key_cp[i], lower_key_cp[i], original[i], step, parameter["Cp_sol"], option=scale_option))
        #Q_t.append(gradient(upper_key_rho[i], lower_key_rho[i], original[i], step, parameter["rho_sol"], option=scale_option))
        Q_t.append(gradient(upper_key_h1[i], lower_key_h1[i],original[i],  step, parameter["delH_1"], option=scale_option))
        Q_t.append(gradient(upper_key_h2[i], lower_key_h2[i],original[i],  step, parameter["delH_2"], option=scale_option))
        Q_t.append(gradient(upper_key_h3[i], lower_key_h3[i],original[i],  step, parameter["delH_3"], option=scale_option))

        Q.append(Q_t)

    if scale_option=="scale_para_and_measure":
        variance = [std*std for i in range(len(original))]
    else:
        variance = [original[i]*original[i]*std*std for i in range(len(original))]
        
    return Q, variance
    
ans_Q, ans_var = Q_way2(measurement_names[0], adsorption=False, scale_option="scale_parameter", step=0.001)

print(np.shape(ans_Q))
print(ans_var)

Measurement:  y_CO2_0
(41, 5)
[1.0000000000000002e-14, 1.3631905588347994e-09, 4.632537744814432e-09, 6.8182243231667885e-09, 3.1926929752802907e-08, 4.4265501659685615e-08, 5.044780565771715e-08, 4.745439354767226e-08, 4.113610804303925e-08, 3.7685455057916684e-08, 3.0913649614247507e-08, 2.7744565431765848e-08, 2.2450655824588337e-08, 2.1327296592944793e-08, 1.9303566473840153e-08, 1.8396838026162353e-08, 1.6764485213412465e-08, 1.5354369441530444e-08, 1.3074293086327036e-08, 1.2605704994055057e-08, 1.1774765892795347e-08, 1.1409864761591351e-08, 1.077110179487426e-08, 1.0249386185011587e-08, 9.479418353075607e-09, 9.221843019428067e-09, 8.883193095712011e-09, 8.797963986187747e-09, 8.735916828689555e-09, 8.757535119512293e-09, 8.864693490032292e-09, 8.949651423882837e-09, 9.155269496841005e-09, 9.275637719093395e-09, 9.534310898961515e-09, 9.672382291515196e-09, 9.95522236987081e-09, 1.0099759351113246e-08, 1.0388365249835088e-08, 1.0417718235068028e-08, 1.044122982348849e-08]


In [13]:
def compute_fim(var_option="std*original", z_pos=[10,20,30,40]):

    fim_overall = np.zeros((len(parameter_names),len(parameter_names)))
    for z in z_pos:
        var = ans_var[z]
        q = np.asarray(ans_Q[z][:]).reshape(len(ans_Q[z][:]), 1)

        fim = 1/var*q@q.T
        
        print("log(e) determinant:", np.log(np.linalg.det(fim)))
        print("log(e) trace:", np.log(np.trace(fim)))
        print("FIM:", fim)
        print("eigenvalue:", np.linalg.eigvals(fim))

        fim_overall += fim 

    print("===total FIM===")
    print("log(e) determinant:", np.log(np.linalg.det(fim_overall)))
    print("log(e) trace:", np.log(np.trace(fim_overall)))
    print("FIM:", fim_overall)

    return fim

compute_fim()

log(e) determinant: -inf
log(e) trace: 61.23025162517421
FIM: [[ 3.90805265e+26  8.88089219e+15 -4.23195090e+13  4.14476240e+13
  -2.30015029e+12]
 [ 8.88089219e+15  2.01814697e+05 -9.61693790e+02  9.41880555e+02
  -5.22699885e+01]
 [-4.23195090e+13 -9.61693790e+02  4.58269373e+00 -4.48827908e+00
   2.49078606e-01]
 [ 4.14476240e+13  9.41880555e+02 -4.48827908e+00  4.39580960e+00
  -2.43946980e-01]
 [-2.30015029e+12 -5.22699885e+01  2.49078606e-01 -2.43946980e-01
   1.35379224e-02]]
eigenvalue: [ 3.90805265e+26 -5.82076609e-11 -1.56839520e-17  1.59336275e-15
  3.33279610e-18]
log(e) determinant: -inf
log(e) trace: 61.35609198031955
FIM: [[ 4.43212681e+26  2.63760273e+16 -3.42437635e+13  3.67089085e+13
  -1.76899515e+12]
 [ 2.63760273e+16  1.56966360e+06 -2.03788041e+03  2.18458364e+03
  -1.05274660e+02]
 [-3.42437635e+13 -2.03788041e+03  2.64576215e+00 -2.83622566e+00
   1.36677162e-01]
 [ 3.67089085e+13  2.18458364e+03 -2.83622566e+00  3.04040029e+00
  -1.46516298e-01]
 [-1.76899515e+

/tmp/ipykernel_330038/1882577787.py:10: RuntimeWarning: divide by zero encountered in log
  print("log(e) determinant:", np.log(np.linalg.det(fim)))
/tmp/ipykernel_330038/1882577787.py:18: RuntimeWarning: invalid value encountered in log
  print("log(e) determinant:", np.log(np.linalg.det(fim_overall)))


array([[ 2.27931001e+25,  7.85917900e+15, -7.25804037e+11,
         8.99065887e+11, -3.75467842e+10],
       [ 7.85917900e+15,  2.70988563e+06, -2.50260992e+02,
         3.10002575e+02, -1.29463257e+01],
       [-7.25804037e+11, -2.50260992e+02,  2.31118846e-02,
        -2.86290872e-02,  1.19560777e-03],
       [ 8.99065887e+11,  3.10002575e+02, -2.86290872e-02,
         3.54633404e-02, -1.48101981e-03],
       [-3.75467842e+10, -1.29463257e+01,  1.19560777e-03,
        -1.48101981e-03,  6.18503404e-05]])

In [14]:
def compute_fim_overall(Q, var, var_option="std*original", z_pos=[10,20,30,40]):

    Q_stack = []
    var_stack = []
    n_z = len(z_pos)
    
    for z in z_pos:
        var_stack.append(var[z])
        #q = np.asarray(ans_Q[z][:]).reshape(len(ans_Q[z][:]), 1)
        qq = Q[z][:]
        Q_stack.append(qq)
        #print(q)

    Q_stack = np.asarray(Q_stack)
    #print(np.shape(Q_stack))
    var_stack = np.asarray(var_stack)

    var = np.zeros((len(z_pos), len(z_pos)))
    for i in range(len(z_pos)):
        var[i,i] = var_stack[i]
    
    inv_var = np.linalg.inv(var)
    #print(var, inv_var)
    fim_overall = Q_stack.T@inv_var@Q_stack
        

    #print("===total FIM===")
    print("log(e) determinant:", np.log(np.linalg.det(fim_overall)))
    print("log(e) trace:", np.log(np.trace(fim_overall)))
    print("FIM:", fim_overall)
    #print("eigenvalue:", np.linalg.eigvals(fim_overall))
    #print("eigen vector:", np.linalg.eig(fim_overall))

    #eigvals = np.linalg.eigvals(fim_overall)
    #eigvec = np.linalg.eig(fim_overall)[1]
    #print(eigvec[:,4])
    #print("t1:", fim_overall@eigvec[:,3])
    #print("t2:", eigvals[3]*eigvec[:,3])

    return fim_overall

#compute_fim_overall(ans_Q, ans_var, z_pos = [10, 20, 30, 40])
#compute_fim_overall(ans_Q, ans_var, z_pos = [5, 15, 25, 35])
#compute_fim_overall(ans_Q, ans_var, z_pos = [5, 15, 25, 35, 10, 20, 30, 40])
#compute_fim_overall(ans_Q, ans_var, z_pos = [5, 10, 15, 20])
#compute_fim_overall(ans_Q, ans_var, z_pos = [25, 30, 35, 40])
compute_fim_overall(ans_Q, ans_var, z_pos = list(range(41)))

log(e) determinant: 78.85456100522467
log(e) trace: 65.1787720581261
FIM: [[ 2.02666068e+28  6.26627934e+17 -7.55612657e+14  8.52982554e+14
  -3.43048331e+13]
 [ 6.26627934e+17  6.46518936e+07 -4.10387663e+04  4.43165732e+04
  -2.10461459e+03]
 [-7.55612657e+14 -4.10387663e+04  7.58034605e+01 -7.59852572e+01
   4.04209570e+00]
 [ 8.52982554e+14  4.43165732e+04 -7.59852572e+01  7.74153708e+01
  -3.98617905e+00]
 [-3.43048331e+13 -2.10461459e+03  4.04209570e+00 -3.98617905e+00
   2.20345997e-01]]


array([[ 2.02666068e+28,  6.26627934e+17, -7.55612657e+14,
         8.52982554e+14, -3.43048331e+13],
       [ 6.26627934e+17,  6.46518936e+07, -4.10387663e+04,
         4.43165732e+04, -2.10461459e+03],
       [-7.55612657e+14, -4.10387663e+04,  7.58034605e+01,
        -7.59852572e+01,  4.04209570e+00],
       [ 8.52982554e+14,  4.43165732e+04, -7.59852572e+01,
         7.74153708e+01, -3.98617905e+00],
       [-3.43048331e+13, -2.10461459e+03,  4.04209570e+00,
        -3.98617905e+00,  2.20345997e-01]])

In [ ]:
def recording_Q(key, discretize=41, store=False):

    all_dict= {}
    all_var = {}
    
    ans_Q, ans_var = Q_way2(key, adsorption=True, scale_option="scale_parameter", step=0.001)
        
    all_dict["ads_"+ key_name] = ans_Q
    all_var["ads_"+ key_name] = ans_var

    ans_Q, ans_var = Q_way2(key, adsorption=False, scale_option="scale_parameter", step=0.001)
        
    all_dict["des_" + key_name] = ans_Q
    all_var["des_"+ key_name] = ans_var

    if store: 
        file = open("./MO_QVs/"+key, "wb")
        pickle.dump(all_dict, file)
        file.close()
        
        file = open("./MO_QVs/Var_"+key, "wb")
        pickle.dump(all_var, file)
        file.close()
        

for c, key_name in enumerate(measurement_names):
    if key_name not in ['qCO2_0', 'qCO2_1', 'qCO2_2', 'Ts_0', 'Ts_1', 'Ts_2', 'ytheta_0', 'ytheta_1', 'ytheta_2']:
        disc = 41
    else:
        disc = 11
    recording_Q(key_name, discretize=disc, store=True)

In [ ]:
def recording_all_fim(key, discretize=41, store=False):

    all_dict= {}
    
    ans_Q, ans_var = Q_way2(key, adsorption=True, scale_option="scale_parameter", step=0.001)
    
    for i in range(discretize):
        key_name = "ads_"+str(i)
        fim_unit = compute_fim_overall(ans_Q, ans_var, z_pos = [i])
        
        all_dict[key_name] = fim_unit

    ans_Q, ans_var = Q_way2(key, adsorption=False, scale_option="scale_parameter", step=0.001)
    
    for i in range(discretize):
        key_name = "des_"+str(i)
        fim_unit = compute_fim_overall(ans_Q, ans_var, z_pos = [i])
        
        all_dict[key_name] = fim_unit

    if store: 
        file = open("./MO_FIMs/"+key, "wb")
        pickle.dump(all_dict, file)
        file.close()

for c, key_name in enumerate(measurement_names):
    if key_name not in ['qCO2_0', 'qCO2_1', 'qCO2_2', 'Ts_0', 'Ts_1', 'Ts_2', 'ytheta_0', 'ytheta_1', 'ytheta_2']:
        disc = 41
    else:
        disc = 11
    recording_all_fim(key_name, discretize=disc, store=True)

In [ ]:
file = open("./MO_FIMs/y_CO2_0", "rb")
data = pickle.load(file)
file.close()

print(np.shape(data["ads_y_CO2_0"]))
print(data["ads_y_CO2_0"])

In [ ]:
def adding_all_fim():

    fim_overall = np.zeros((5,5))

    for key in measurement_names:
        ans_Q, ans_var = Q_way2(key, adsorption=True, scale_option="not_scale", step=0.001)
        try:
            fim_unit = compute_fim_overall(ans_Q, ans_var, z_pos = list(range(41)))
        except:
            fim_unit = compute_fim_overall(ans_Q, ans_var, z_pos = list(range(11)))
        fim_overall += fim_unit

    for key in measurement_names:
        ans_Q, ans_var = Q_way2(key, adsorption=False, scale_option="not_scale", step=0.001)
        try:
            fim_unit = compute_fim_overall(ans_Q, ans_var, z_pos = list(range(41)))
        except:
            fim_unit = compute_fim_overall(ans_Q, ans_var, z_pos = list(range(11)))
        fim_overall += fim_unit
        

    print("===total FIM===")
    print("log(e) determinant:", np.log(np.linalg.det(fim_overall)))
    print("log(e) trace:", np.log(np.trace(fim_overall)))
    print("FIM:", fim_overall)
    print("eigenvalue:", np.linalg.eigvals(fim_overall))


adding_all_fim()

In [ ]:
# compute 
def gradient(key):
    upper_ads_key = upper_ads[key]
    lower_ads_key = lower_ads[key]

    upper_des_key = upper_des[key]
    lower_des_key = lower_des[key]

    #print(upper_ads_key)
    #print(lower_ads_key)
    ads_finite = 0 
    for i in range(len(upper_ads_key)):
        ads_finite += (upper_ads_key[i] - lower_ads_key[i])/2/step/param_value
        #print(ads_finite)

    des_finite = 0 
    for i in range(len(upper_des_key)):
        des_finite += (upper_des_key[i] - lower_des_key[i])/2/step/param_value
    
    print("Measurement: ", key, " adsorption, gradient:", ads_finite)
    print("Measurement: ", key, " desorption, gradient:", des_finite)
    
    return ads_finite, des_finite


In [ ]:
ads_gradient, des_gradient = [], []

for n in measurement_names:

    ads_f, des_f = gradient(n)
    ads_gradient.append(ads_f)
    des_gradient.append(des_f)


print(ads_gradient, des_gradient)

In [ ]:
def draw_all_var():
    plt.plot(range(len(ads_gradient)), des_gradient, "o")
    #plt.xlabel("Axial node")
    plt.xticks(range(27) ,measurement_names, rotation=60)
    plt.ylabel("Desorption model gradient")
    plt.show() 

draw_all_var()

In [ ]:
# draw

def draw(key, upper_ads, lower_ads):
    upper_ads_key = upper_ads[key]
    lower_ads_key = lower_ads[key]
    print(upper_ads_key)
    print(lower_ads_key)

    #upper_des_key = upper_des[key]
    #lower_des_key = lower_des[key]

    plt.plot(range(len(upper_ads_key)), upper_ads_key, "-o", label="delH_1=98.76")
    plt.plot(range(len(lower_ads_key)), lower_ads_key, "-*", label="delH_1=10000")
    plt.xlabel("Axial node")
    plt.ylabel(key+" Adsorption")
    plt.legend()
    plt.show() 

    #plt.plot(range(len(upper_des_key)), upper_des_key, "-o", label="Upper perturbation")
    #plt.plot(range(len(lower_des_key)), lower_des_key, "-*", label="lower perturbation")
    #plt.xlabel("Axial node")
    #plt.ylabel(key+" Desorption")
    #plt.legend()
    #plt.show() 
    
    plt.plot()
    

In [ ]:
for n in measurement_names:
    draw(n, upper_ads_h1, lower_ads_h1)

In [11]:
rank = {('Tg_0', 5, 0): 0.007087903731182191, ('Tg_0', 5, 1): 0.9212552144447199, ('Tg_0', 20, 0): 0.001083641922875856, ('Tg_0', 20, 1): 0.7170849675804832, ('Tg_0', 40, 0): 0.0006913308345041581, ('Tg_0', 40, 1): 0.7671467359282383, ('Tg_1', 5, 0): 17.690657180335993, ('Tg_1', 5, 1): 25.554644897287673, ('Tg_1', 20, 0): 0.5904076968923306, ('Tg_1', 20, 1): 1.266768553884345, ('Tg_1', 40, 0): 0.47926084000542507, ('Tg_1', 40, 1): 1.1484148334934279, ('Tg_2', 5, 0): 279.7630877805168, ('Tg_2', 5, 1): 274.9794329975756, ('Tg_2', 20, 0): 0.9437807299467917, ('Tg_2', 20, 1): 3.1542022338566467, ('Tg_2', 40, 0): 0.9127656858144447, ('Tg_2', 40, 1): 3.525528063012056, ('Pg_0', 5, 0): 0.00023211400390898417, ('Pg_0', 5, 1): 0.0008459461190341571, ('Pg_0', 20, 0): 0.0056736514774814555, ('Pg_0', 20, 1): 0.004821869695517253, ('Pg_0', 40, 0): 0.0, ('Pg_0', 40, 1): 0.0, ('Pg_1', 5, 0): 0.0021556956300519815, ('Pg_1', 5, 1): 0.0020098683592683136, ('Pg_1', 20, 0): 0.0008838740425755533, ('Pg_1', 20, 1): 0.001488447301239255, ('Pg_1', 40, 0): 0.0, ('Pg_1', 40, 1): 0.0, ('Pg_2', 5, 0): 0.00048114463716151325, ('Pg_2', 5, 1): 0.0007118816214281271, ('Pg_2', 20, 0): 0.028927505334822443, ('Pg_2', 20, 1): 0.029526749061106076, ('Pg_2', 40, 0): 0.0, ('Pg_2', 40, 1): 0.0, ('vg_0', 5, 0): 2.945600731224929, ('vg_0', 5, 1): 1.6058071508996665, ('vg_0', 20, 0): 0.1180310433181826, ('vg_0', 20, 1): 0.08453899126354555, ('vg_0', 40, 0): 4.035121631987036, ('vg_0', 40, 1): 2.0642062909857515, ('vg_1', 5, 0): 16.52212975451086, ('vg_1', 5, 1): 15.465576768245453, ('vg_1', 20, 0): 0.12161918109763517, ('vg_1', 20, 1): 0.09382712196914131, ('vg_1', 40, 0): 0.34602545177085914, ('vg_1', 40, 1): 1.1311470481181913, ('vg_2', 5, 0): 330.2783032969805, ('vg_2', 5, 1): 179.03175413299584, ('vg_2', 20, 0): 0.673842740204949, ('vg_2', 20, 1): 0.31498739568919315, ('vg_2', 40, 0): 14.974661228064267, ('vg_2', 40, 1): 7.601285674494682, ('Ts_0', 1, 0): 0.002817359343491693, ('Ts_0', 1, 1): 0.05816116657777877, ('Ts_0', 5, 0): 238.2873964314901, ('Ts_0', 5, 1): 235.47483668064763, ('Ts_0', 10, 0): 0.3962108955125392, ('Ts_0', 10, 1): 0.3633375246972235, ('Ts_1', 1, 0): 6.831757042e-05, ('Ts_1', 1, 1): 0.8256528593866788, ('Ts_1', 5, 0): 0.3473821422396362, ('Ts_1', 5, 1): 0.4367084875694609, ('Ts_1', 10, 0): 0.9232089790675809, ('Ts_1', 10, 1): 3.2346544171253293, ('Ts_2', 1, 0): 4.3262906827544446e-05, ('Ts_2', 1, 1): 0.8751631190432193, ('Ts_2', 5, 0): 0.2717572039694804, ('Ts_2', 5, 1): 0.3302289306591711, ('Ts_2', 10, 0): 0.912372844791925, ('Ts_2', 10, 1): 3.5300058183112717, ('y_CO2_0', 5, 0): 438097.69222117285, ('y_CO2_0', 5, 1): 19663.709039399386, ('y_CO2_0', 20, 0): 31454.1496254842, ('y_CO2_0', 20, 1): 47778.00778954663, ('y_CO2_0', 40, 0): 311.511241547329, ('y_CO2_0', 40, 1): 2279.653454445296, ('y_CO2_1', 5, 0): 7930.3741644063075, ('y_CO2_1', 5, 1): 4486.157010459178, ('y_CO2_1', 20, 0): 4659.1423350480445, ('y_CO2_1', 20, 1): 1490.2903798022242, ('y_CO2_1', 40, 0): 1314.1541078570915, ('y_CO2_1', 40, 1): 471.8646937618247, ('y_CO2_2', 5, 0): 30225.020321498505, ('y_CO2_2', 5, 1): 384919.9706731455, ('y_CO2_2', 20, 0): 71099.90809911578, ('y_CO2_2', 20, 1): 8065.839464904935, ('y_CO2_2', 40, 0): 1996.8623958681596, ('y_CO2_2', 40, 1): 966.2326465411189, ('C_CO2_0', 5, 0): 437778.54466704436, ('C_CO2_0', 5, 1): 21372.855194603515, ('C_CO2_0', 20, 0): 31408.10578536428, ('C_CO2_0', 20, 1): 46414.685372850865, ('C_CO2_0', 40, 0): 310.9334277660297, ('C_CO2_0', 40, 1): 2180.7446206538752, ('C_CO2_1', 5, 0): 8724.884872381204, ('C_CO2_1', 5, 1): 4637.754009338893, ('C_CO2_1', 20, 0): 4613.712175740087, ('C_CO2_1', 20, 1): 1498.7458401786228, ('C_CO2_1', 40, 0): 1269.7215469398093, ('C_CO2_1', 40, 1): 439.7166941180052, ('C_CO2_2', 5, 0): 31335.36487344622, ('C_CO2_2', 5, 1): 406410.5259296241, ('C_CO2_2', 20, 0): 70518.29571251266, ('C_CO2_2', 20, 1): 8631.306893423252, ('C_CO2_2', 40, 0): 1904.552154416097, ('C_CO2_2', 40, 1): 886.4700730425178, ('C_N2_0', 5, 0): 12.909491488997691, ('C_N2_0', 5, 1): 8402993.24648279, ('C_N2_0', 20, 0): 5.823929292906303, ('C_N2_0', 20, 1): 4379397.555852882, ('C_N2_0', 40, 0): 0.10060268157259701, ('C_N2_0', 40, 1): 37107.022664874065, ('C_N2_1', 5, 0): 22.490659659699162, ('C_N2_1', 5, 1): 14927338.744959986, ('C_N2_1', 20, 0): 1.1056966345022523, ('C_N2_1', 20, 1): 846.0641240645206, ('C_N2_1', 40, 0): 1.2284292847935712, ('C_N2_1', 40, 1): 55528.85044379615, ('C_N2_2', 5, 0): 661.4912732610305, ('C_N2_2', 5, 1): 527485927.54296917, ('C_N2_2', 20, 0): 10.0681996105036, ('C_N2_2', 20, 1): 7791423.235700663, ('C_N2_2', 40, 0): 1.9272182895548275, ('C_N2_2', 40, 1): 18798.379867726453, ('qCO2_0', 1, 0): 168343.31960595932, ('qCO2_0', 1, 1): 4318.8782744631, ('qCO2_0', 5, 0): 68387.23871711528, ('qCO2_0', 5, 1): 63792.77655430321, ('qCO2_0', 10, 0): 10076.600994067672, ('qCO2_0', 10, 1): 487192.6738510713, ('qCO2_1', 1, 0): 20.26169674608753, ('qCO2_1', 1, 1): 28.479819865580424, ('qCO2_1', 5, 0): 51.418568795992954, ('qCO2_1', 5, 1): 50.807092390372894, ('qCO2_1', 10, 0): 16.92510658579889, ('qCO2_1', 10, 1): 9.422160234176523, ('qCO2_2', 1, 0): 2.839122008213145, ('qCO2_2', 1, 1): 3.5229720337359094, ('qCO2_2', 5, 0): 5.521304935673507, ('qCO2_2', 5, 1): 7.264887709661095, ('qCO2_2', 10, 0): 9.111884235279842, ('qCO2_2', 10, 1): 8.316653615454767, ('ytheta_0', 1, 0): 12657.757298712986, ('ytheta_0', 1, 1): 68564.2293233587, ('ytheta_0', 5, 0): 63145.86081972869, ('ytheta_0', 5, 1): 346231.6402065682, ('ytheta_0', 10, 0): 4.513898307157586e-22, ('ytheta_0', 10, 1): 6.018531219703076e-15, ('ytheta_1', 1, 0): 23767.82573414949, ('ytheta_1', 1, 1): 45694.768486645175, ('ytheta_1', 5, 0): 2824.02005818814, ('ytheta_1', 5, 1): 1420.474659961933, ('ytheta_1', 10, 0): 2.181717515126164e-21, ('ytheta_1', 10, 1): 2.2870420672471734e-14, ('ytheta_2', 1, 0): 397.5991819500919, ('ytheta_2', 1, 1): 2824.8734958405657, ('ytheta_2', 5, 0): 1034.2574355656982, ('ytheta_2', 5, 1): 347.2761137437938, ('ytheta_2', 10, 0): 5.040519776325972e-21, ('ytheta_2', 10, 1): 3.129638175705383e-14}

In [28]:
print(rank[('C_CO2_0', 5, 0)])
print(rank[('Pg_2', 5, 1)])
print(rank[('vg_0', 40, 0)])
print(rank[('y_CO2_1', 40, 1)])

437778.54466704436
0.0007118816214281271
4.035121631987036
471.8646937618247


In [27]:
sorted_rank = sorted(rank, key=rank.get, reverse=True)

print(sorted_rank)

[('C_N2_2', 5, 1), ('C_N2_1', 5, 1), ('C_N2_0', 5, 1), ('C_N2_2', 20, 1), ('C_N2_0', 20, 1), ('qCO2_0', 10, 1), ('y_CO2_0', 5, 0), ('C_CO2_0', 5, 0), ('C_CO2_2', 5, 1), ('y_CO2_2', 5, 1), ('ytheta_0', 5, 1), ('qCO2_0', 1, 0), ('y_CO2_2', 20, 0), ('C_CO2_2', 20, 0), ('ytheta_0', 1, 1), ('qCO2_0', 5, 0), ('qCO2_0', 5, 1), ('ytheta_0', 5, 0), ('C_N2_1', 40, 1), ('y_CO2_0', 20, 1), ('C_CO2_0', 20, 1), ('ytheta_1', 1, 1), ('C_N2_0', 40, 1), ('y_CO2_0', 20, 0), ('C_CO2_0', 20, 0), ('C_CO2_2', 5, 0), ('y_CO2_2', 5, 0), ('ytheta_1', 1, 0), ('C_CO2_0', 5, 1), ('y_CO2_0', 5, 1), ('C_N2_2', 40, 1), ('ytheta_0', 1, 0), ('qCO2_0', 10, 0), ('C_CO2_1', 5, 0), ('C_CO2_2', 20, 1), ('y_CO2_2', 20, 1), ('y_CO2_1', 5, 0), ('y_CO2_1', 20, 0), ('C_CO2_1', 5, 1), ('C_CO2_1', 20, 0), ('y_CO2_1', 5, 1), ('qCO2_0', 1, 1), ('ytheta_2', 1, 1), ('ytheta_1', 5, 0), ('y_CO2_0', 40, 1), ('C_CO2_0', 40, 1), ('y_CO2_2', 40, 0), ('C_CO2_2', 40, 0), ('C_CO2_1', 20, 1), ('y_CO2_1', 20, 1), ('ytheta_1', 5, 1), ('y_CO2_1', 

In [29]:
rank_ratio = rank.copy()

In [30]:
for n in rank_ratio:
    if n[0] in ["Tg_0", "Tg_1", "Tg_2",
                "Pg_0", "Pg_1","Pg_2",
                "vg_0", "vg_1","vg_2",
                "Ts_0", "Ts_1","Ts_2",]:
        rank_ratio[n] = rank_ratio[n]/10
        
#print(rank_ratio)

In [31]:
sorted_rank_ratio = sorted(rank_ratio, key=rank_ratio.get, reverse=True)
print(sorted_rank_ratio)

[('C_N2_2', 5, 1), ('C_N2_1', 5, 1), ('C_N2_0', 5, 1), ('C_N2_2', 20, 1), ('C_N2_0', 20, 1), ('qCO2_0', 10, 1), ('y_CO2_0', 5, 0), ('C_CO2_0', 5, 0), ('C_CO2_2', 5, 1), ('y_CO2_2', 5, 1), ('ytheta_0', 5, 1), ('qCO2_0', 1, 0), ('y_CO2_2', 20, 0), ('C_CO2_2', 20, 0), ('ytheta_0', 1, 1), ('qCO2_0', 5, 0), ('qCO2_0', 5, 1), ('ytheta_0', 5, 0), ('C_N2_1', 40, 1), ('y_CO2_0', 20, 1), ('C_CO2_0', 20, 1), ('ytheta_1', 1, 1), ('C_N2_0', 40, 1), ('y_CO2_0', 20, 0), ('C_CO2_0', 20, 0), ('C_CO2_2', 5, 0), ('y_CO2_2', 5, 0), ('ytheta_1', 1, 0), ('C_CO2_0', 5, 1), ('y_CO2_0', 5, 1), ('C_N2_2', 40, 1), ('ytheta_0', 1, 0), ('qCO2_0', 10, 0), ('C_CO2_1', 5, 0), ('C_CO2_2', 20, 1), ('y_CO2_2', 20, 1), ('y_CO2_1', 5, 0), ('y_CO2_1', 20, 0), ('C_CO2_1', 5, 1), ('C_CO2_1', 20, 0), ('y_CO2_1', 5, 1), ('qCO2_0', 1, 1), ('ytheta_2', 1, 1), ('ytheta_1', 5, 0), ('y_CO2_0', 40, 1), ('C_CO2_0', 40, 1), ('y_CO2_2', 40, 0), ('C_CO2_2', 40, 0), ('C_CO2_1', 20, 1), ('y_CO2_1', 20, 1), ('ytheta_1', 5, 1), ('y_CO2_1', 